In [3]:
ws = Workspace.from_config()

In [4]:
exp = Experiment(workspace=ws, name="udacity-project")

run = exp.start_logging()

In [5]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
cpu_cluster_name = 'UdacityProject1'

compute_config = AmlCompute.provisioning_configuration(vm_size = 'Standard_D2_V2',max_nodes = 4)

cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output = True)

Creating......
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [35]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
ps = RandomParameterSampling({
        "number_of_hidden_layers": choice(range(1,5)),
        "batch_size": choice(16, 32, 64, 128)
    }
)

# Specify a Policy
policy = policy = BanditPolicy(slack_factor=0.15, evaluation_interval=1, delay_evaluation=10)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
# # pip_packages = [
#                 'azureml-core==1.17.0', 'azureml-telemetry==1.17.0', 'azureml-dataprep==2.4.2',
#                 'joblib==0.14.1', 'pandas==1.0.0', 'sklearn-pandas==2.0.2' 
#                ]

est = SKLearn(source_directory='.', 
            compute_target=cpu_cluster,
            entry_script='train.py',
#             pip_packages=pip_packages,
            conda_packages=['scikit-learn==0.22.2.post1'])



In [36]:
# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                             hyperparameter_sampling=ps,
                             policy=policy,
                             primary_metric_name="Accuracy",
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=100,
                             max_concurrent_runs=4)

In [37]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = exp.submit(hyperdrive_config)

In [38]:
from azureml.widgets import RunDetails
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [ ]:
import joblib
# Get your best run and save the model from that run.

In [52]:
hyperdrive_run.get_metrics()

{}

In [58]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
#       ['runDefinition']['arguments'])

AttributeError: 'NoneType' object has no attribute 'get_metrics'

In [57]:
parameter_value = best_run.get_details()['runDefinition']['arguments']

AttributeError: 'NoneType' object has no attribute 'get_details'

In [46]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
path = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
autods = TabularDatasetFactory.from_delimited_files(path = path)

In [47]:
from train import clean_data

# Use the clean_data function to clean your data.
ds = clean_data(autods)

In [48]:
x = ds
y = x.pop('y_cleaned')

In [49]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=223)

In [50]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='Classification',
    primary_metric='Accuracy',
    training_data=x_train,
    label_column_name=y_train,
    n_cross_validations=5)

ConfigException: ConfigException:
	Message: Invalid argument(s) 'Classification' specified. Supported value(s): '['classification', 'regression', 'forecasting', 'image-classification', 'image-multi-labeling', 'image-object-detection', 'image-instance-segmentation', 'text-classification-multilabel']'.
	InnerException: None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "Invalid argument(s) 'Classification' specified. Supported value(s): '['classification', 'regression', 'forecasting', 'image-classification', 'image-multi-labeling', 'image-object-detection', 'image-instance-segmentation', 'text-classification-multilabel']'.",
        "details_uri": "https://aka.ms/AutoMLConfig",
        "target": "task_type",
        "inner_error": {
            "code": "BadArgument",
            "inner_error": {
                "code": "ArgumentInvalid"
            }
        }
    }
}

In [2]:
# Submit your automl run

run = exp.submit(config = automl_config, show_output = True)

In [ ]:
# Retrieve and save your best automl model.

best_run = AutoMLConfig.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments'])